In [110]:
with open("inputs/Day_20.txt") as f:
   input_data = f.read()

In [125]:
def part_1_solution(raw_input):
    maze = parse_maze(raw_input)
    
    start_position = list(maze['AA'])[0]
    reachable_positions = bfs(maze, start_position)
    
    target_position = list(maze['ZZ'])[0]
    path_to_ZZ = get_path_to(target_position, start_position, reachable_positions)
    
    return len(path_to_ZZ) - 1 # don't count start tile 

    
def parse_maze(raw_input):
    maze = dict()
    processed = set()
    raw_maze_by_lines = raw_input.split("\n")
    
    for y, row in enumerate(raw_maze_by_lines):
        for x, obj in enumerate(row):
            position = (x, y)
            if obj == "#" or obj == " ":
                continue 
                
            if position in processed:
                continue
                
            if obj == ".":
                maze[position] = "OPEN_PASSAGE"
                continue
                
            # must be a portal or start/end
            
            # find portal second letter
            for x_offset, y_offset in ((1, 0), (0, 1)):
                cand_x, cand_y = position[0] + x_offset, position[1] + y_offset
                
                cand_tile = raw_maze_by_lines[cand_y][cand_x] 
                
                if cand_tile.isupper():
                    # found second letter
                    portal_name = obj + cand_tile
                    second_letter_position = (cand_x, cand_y)
                    processed.add(second_letter_position)
                    break
                    
            for base_x, base_y in (position, second_letter_position):
                for x_offset, y_offset in ((0, -1), (1, 0), (0, 1), (-1, 0)):
                    cand_x, cand_y = base_x + x_offset, base_y + y_offset
                    
                    try:
                        cand_tile = raw_maze_by_lines[cand_y][cand_x]
                    except:
                        continue
                        
                    if cand_tile == ".":
                        # found tile connected to portal
                        portal_position = (cand_x, cand_y)
                        processed.add(portal_position)
                        maze[portal_position] = portal_name
                        
                        if portal_name not in maze:
                            maze[portal_name] = set()
                            
                        maze[portal_name].add(portal_position)
    return maze

def bfs(maze, start_position):
    visited = set()
    path_to = dict()
    
    queue = list()
    queue.append(start_position)
    visited.add(start_position)
    
    while queue:
        current_position = queue.pop(0)
        
        for neighbor in get_neighbors(current_position, maze):
            if neighbor not in visited:
                visited.add(neighbor)
                path_to[neighbor] = current_position
                
                queue.append(neighbor)
                
    return path_to
    
def get_neighbors(current_position, maze):
    
    current_tile = maze[current_position]
    
    if current_tile != "OPEN_PASSAGE":
        # portal or start/end
        portal_positions = maze[current_tile]

        if len(portal_positions) > 1:
            # portal
            next_portal_position = list(portal_positions - {current_position})[0]
            yield next_portal_position
    
    for x_offset, y_offset in ((0, -1), (1, 0), (0, 1), (-1, 0)):
        candidate = (current_position[0] + x_offset, current_position[1] + y_offset)

        if candidate in maze:
            yield candidate
            
            
def get_path_to(target_position, start_position, bfs_path_to):
    path = list()
    current_position = target_position
    while current_position != start_position:
        path.append(current_position)
        current_position = bfs_path_to[current_position]
        
    path.append(start_position)
    path.reverse()
    
    return path 

In [126]:
test_input = """         A           
         A           
  #######.#########  
  #######.........#  
  #######.#######.#  
  #######.#######.#  
  #######.#######.#  
  #####  B    ###.#  
BC...##  C    ###.#  
  ##.##       ###.#  
  ##...DE  F  ###.#  
  #####    G  ###.#  
  #########.#####.#  
DE..#######...###.#  
  #.#########.###.#  
FG..#########.....#  
  ###########.#####  
             Z       
             Z       """
assert(part_1_solution(test_input) == 23)
print("Test passed")

Test passed


In [114]:
test_input = """                   A               
                   A               
  #################.#############  
  #.#...#...................#.#.#  
  #.#.#.###.###.###.#########.#.#  
  #.#.#.......#...#.....#.#.#...#  
  #.#########.###.#####.#.#.###.#  
  #.............#.#.....#.......#  
  ###.###########.###.#####.#.#.#  
  #.....#        A   C    #.#.#.#  
  #######        S   P    #####.#  
  #.#...#                 #......VT
  #.#.#.#                 #.#####  
  #...#.#               YN....#.#  
  #.###.#                 #####.#  
DI....#.#                 #.....#  
  #####.#                 #.###.#  
ZZ......#               QG....#..AS
  ###.###                 #######  
JO..#.#.#                 #.....#  
  #.#.#.#                 ###.#.#  
  #...#..DI             BU....#..LF
  #####.#                 #.#####  
YN......#               VT..#....QG
  #.###.#                 #.###.#  
  #.#...#                 #.....#  
  ###.###    J L     J    #.#.###  
  #.....#    O F     P    #.#...#  
  #.###.#####.#.#####.#####.###.#  
  #...#.#.#...#.....#.....#.#...#  
  #.#####.###.###.#.#.#########.#  
  #...#.#.....#...#.#.#.#.....#.#  
  #.###.#####.###.###.#.#.#######  
  #.#.........#...#.............#  
  #########.###.###.#############  
           B   J   C               
           U   P   P               """
assert(part_1_solution(test_input) == 58)
print("Test passed")

Test passed


In [115]:
print(f"Part 1 solution: {part_1_solution(input_data)}")

Part 1 solution: 690


In [199]:
def part_2_solution(raw_input):
    grid = parse_grid(raw_input)
    
    reachable_positions = bfs(grid)
    
    path_to_ZZ = get_path_to(grid['end'], grid['start'], reachable_positions)
    
    return len(path_to_ZZ) - 1 # don't count start tile 

    
def parse_grid(raw_input):
    maze = dict()
    processed = set()
    start_position = None
    end_position = None
    portal = dict()
    raw_maze_by_lines = raw_input.split("\n")
    min_x = len(raw_maze_by_lines[0])
    max_x = 0
    
    min_y = len(raw_maze_by_lines)
    max_y = 0
    
    for y, row in enumerate(raw_maze_by_lines):
        for x, obj in enumerate(row):
            position = (x, y)
            if obj == "#":
                min_x = min(x, min_x)
                max_x = max(x, max_x)
                
                min_y = min(y, min_y)
                max_y = max(y, max_y)
                continue 
                
            if obj == " ":
                continue
                
            if position in processed:
                continue
                
            if obj == ".":
                maze[position] = "OPEN_PASSAGE"
                continue
                
            # must be a portal or start/end
            
            # find portal second letter
            for x_offset, y_offset in ((1, 0), (0, 1)):
                cand_x, cand_y = position[0] + x_offset, position[1] + y_offset
                
                cand_tile = raw_maze_by_lines[cand_y][cand_x] 
                
                if cand_tile.isupper():
                    # found second letter
                    portal_name = obj + cand_tile
                    second_letter_position = (cand_x, cand_y)
                    processed.add(second_letter_position)
                    break
                    
            for base_x, base_y in (position, second_letter_position):
                for x_offset, y_offset in ((0, -1), (1, 0), (0, 1), (-1, 0)):
                    cand_x, cand_y = base_x + x_offset, base_y + y_offset
                    
                    try:
                        cand_tile = raw_maze_by_lines[cand_y][cand_x]
                    except:
                        continue
                        
                    if cand_tile == ".":
                        # found tile connected to portal
                        portal_position = (cand_x, cand_y)
                        processed.add(portal_position)
                        maze[portal_position] = portal_name
                        
                        if portal_name not in portal:
                            portal[portal_name] = set()
                            
                        portal[portal_name].add(portal_position)
    
    
    start_position = list(portal['AA'])[0]
    end_position = list(portal['ZZ'])[0]
    
    # find outer portals
    positions = list(maze.keys())
    
    outer_portals = set()
    
    for position in positions:
        x, y = position
        if x == min_x or x == max_x or y == min_y or y == max_y:
            if position == start_position or position == end_position:
                continue
            
            outer_portals.add(position)
    
    grid = {
        'maze': maze,
        'start': start_position,
        'end': end_position,
        'outer_portals': outer_portals,
        'portals': portal
    }
    
    return grid

def bfs(grid):
    visited = dict()
    path_to = dict()
    current_lvl = 0
    start_position = grid['start']
    end_position = grid['end']
    
    queue = list()
    queue.append((start_position, current_lvl))
    
    visited[current_lvl] = set()
    visited[current_lvl].add(start_position)
    
    while queue:
        current_position, current_lvl = queue.pop(0)
        
        for neighbor, next_lvl in get_neighbors(current_position, current_lvl, grid):
            if next_lvl not in visited:
                visited[next_lvl] = set()
            
            if neighbor not in visited[next_lvl]:
                visited[next_lvl].add(neighbor)
                path_to[(neighbor, next_lvl)] = (current_position, current_lvl)
                
                if next_lvl == 0 and neighbor == end_position:
                    # found path to end position
                    return path_to
                
                queue.append((neighbor, next_lvl))
    
    # path to end_position not found
    raise Exception(f"Path to {end_position} not found")
    
    
def get_neighbors(current_position, current_lvl, grid):
    
    neighbors = list()
    
    maze = grid['maze']
    current_tile = maze[current_position]
    
    if current_tile != "OPEN_PASSAGE":
        # portal or start/end
        if current_lvl == 0:
            if current_position in grid['outer_portals']:
                # on 0 lvl outer portal is a dead-end
                return list()
        else:
            if current_position == grid['start'] or current_position == grid['end']:
                return list()
        
        portal_positions = grid['portals'][current_tile]
        
        if len(portal_positions) > 1:
            next_portal_position = list(portal_positions - {current_position})[0]
            if current_position in grid['outer_portals']:
                # go one lvl up
                neighbors.append((next_portal_position, current_lvl - 1))
            else:
                # go one lvl down (inner portal)
                neighbors.append((next_portal_position, current_lvl + 1))
    
    for x_offset, y_offset in ((0, -1), (1, 0), (0, 1), (-1, 0)):
        candidate = (current_position[0] + x_offset, current_position[1] + y_offset)

        if candidate in maze:
            neighbors.append((candidate, current_lvl))
            
    return neighbors
            
            
def get_path_to(target_position, start_position, bfs_path_to):
    path = list()
    current_position = target_position
    current_lvl = 0
    while current_position != start_position or current_lvl != 0:
        path.append((current_position, current_lvl))
        current_position, current_lvl = bfs_path_to[(current_position, current_lvl)]
        
    path.append((start_position, 0))
    path.reverse()
    
    return path 

In [200]:
test_input = """         A           
         A           
  #######.#########  
  #######.........#  
  #######.#######.#  
  #######.#######.#  
  #######.#######.#  
  #####  B    ###.#  
BC...##  C    ###.#  
  ##.##       ###.#  
  ##...DE  F  ###.#  
  #####    G  ###.#  
  #########.#####.#  
DE..#######...###.#  
  #.#########.###.#  
FG..#########.....#  
  ###########.#####  
             Z       
             Z       """
assert(part_2_solution(test_input) == 26)
print("Test passed")

Test passed


In [201]:
test_input = """             Z L X W       C                 
             Z P Q B       K                 
  ###########.#.#.#.#######.###############  
  #...#.......#.#.......#.#.......#.#.#...#  
  ###.#.#.#.#.#.#.#.###.#.#.#######.#.#.###  
  #.#...#.#.#...#.#.#...#...#...#.#.......#  
  #.###.#######.###.###.#.###.###.#.#######  
  #...#.......#.#...#...#.............#...#  
  #.#########.#######.#.#######.#######.###  
  #...#.#    F       R I       Z    #.#.#.#  
  #.###.#    D       E C       H    #.#.#.#  
  #.#...#                           #...#.#  
  #.###.#                           #.###.#  
  #.#....OA                       WB..#.#..ZH
  #.###.#                           #.#.#.#  
CJ......#                           #.....#  
  #######                           #######  
  #.#....CK                         #......IC
  #.###.#                           #.###.#  
  #.....#                           #...#.#  
  ###.###                           #.#.#.#  
XF....#.#                         RF..#.#.#  
  #####.#                           #######  
  #......CJ                       NM..#...#  
  ###.#.#                           #.###.#  
RE....#.#                           #......RF
  ###.###        X   X       L      #.#.#.#  
  #.....#        F   Q       P      #.#.#.#  
  ###.###########.###.#######.#########.###  
  #.....#...#.....#.......#...#.....#.#...#  
  #####.#.###.#######.#######.###.###.#.#.#  
  #.......#.......#.#.#.#.#...#...#...#.#.#  
  #####.###.#####.#.#.#.#.###.###.#.###.###  
  #.......#.....#.#...#...............#...#  
  #############.#.#.###.###################  
               A O F   N                     
               A A D   M   """
assert(part_2_solution(test_input) == 396)
print("Test passed")

Test passed


In [202]:
print(f"Part 2 solution: {part_2_solution(input_data)}")

Part 2 solution: 7976
